In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(256, input_dim=X_train.shape[1], kernel_initializer=init, activation='relu'))
    model.add(Dense(128, kernel_initializer=init, activation='relu'))
    model.add(Dense(64, kernel_initializer=init, activation='relu'))
    model.add(Dense(32, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(model=create_model, verbose=1)
print(model.get_params().keys())
print(model.get_params())

In [ ]:
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150,200]
batches = [5, 10, 20]

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, model__init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [ ]:
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=10, cv=kfold, scoring='accuracy', verbose=1, n_jobs=-1)
random_search_results = random_search.fit(X_train, y_train)

print(f"Best Accuracy: {random_search_results.best_score_:.2f}%")
print(f"Best Parameters: {random_search_results.best_params_}")

In [ ]:
# Neural network with optimal hyperparameters
best_model = random_search_results.best_estimator_
best_model.fit(X_train, y_train, epochs=best_model.epochs, batch_size=best_model.batch_size, verbose=1, validation_split=0.2)
best_scores = best_model.model.evaluate(X_test, y_test, verbose=0)
print(f"Best Model Accuracy: {best_scores[1]*100:.2f}%")

In [ ]:
# Cross-validation with the best model
cv_scores = []
for train, val in kfold.split(X_train, y_train):
    cv_model = create_model(learning_rate=best_model.learning_rate, dropout_rate=best_model.dropout_rate, weight_decay=best_model.weight_decay)
    cv_model.fit(X_train[train], y_train[train], epochs=best_model.epochs, batch_size=best_model.batch_size, verbose=0)
    _, accuracy = cv_model.evaluate(X_train[val], y_train[val], verbose=0)
    cv_scores.append(accuracy)

print(f"Cross-Validation Accuracy: {np.mean(cv_scores)*100:.2f}%")